In [2]:
import pandas as pd

In [4]:
subset1_pat = pd.read_json('../../Annotation/Patrick/batch1_patrick_result.json')
subset2_pat = pd.read_json('../../Annotation/Patrick/batch2_patrick_result.json')
subset3_pat = pd.read_json('../../Annotation/Patrick/batch3_patrick_result.json')
subset1_thom = pd.read_json('../../Annotation/Thom/batch1_thom_result.json')
subset1_chiel = pd.read_json('../../Annotation/Chiel/batch1_chiel_result.json')
subset2_chiel = pd.read_json('../../Annotation/Chiel/batch2_chiel_result.json')
subset3_chiel = pd.read_json('../../Annotation/Chiel/batch3_chiel_result.json')
subset1_bar = pd.read_json('batch1_result.json')
subset2_bar = pd.read_json('result.json')

In [14]:
subset3_chiel

,uuid,rubriek,notaris,inventarisNr,akteNr,akteType,datering,taal,beschrijving,namen,urls,text,name,data_matches,data_entry,vocop_match
5000,6a8c2759-a8a2-739c-7c88-66cc2b3074b9,358,JAN VERLEIJ,11945,26354,Testament,1761-06-24,nederlands,None,"[{'voornaam': 'Magteld', 'tussenvoegsel': None...","['KLAB06150000131.JPG', 'KLAB06150000132.JPG',...",160 Mgendes Dit Testament van Man No: 7160 en ...,Hendrik Verhoeve,"[{'index': 83119, 'name_original': 'Hendrikus ...","{'83119': {'ships': ['nan'], 'rank': [], 'loca...",0
5001,6a8c2759-a8a2-739c-7c88-66cc2b3074b9,358,JAN VERLEIJ,11945,26354,Testament,1761-06-24,nederlands,None,"[{'voornaam': 'Magteld', 'tussenvoegsel': None...","['KLAB06150000131.JPG', 'KLAB06150000132.JPG',...",160 Mgendes Dit Testament van Man No: 7160 en ...,Jan Gielink,"[{'index': 69002, 'name_original': 'Jan Geleij...","{'69002': {'ships': [], 'rank': [], 'location'...",0
5002,6a8c2759-a8a2-739c-7c88-66cc2b3074b9,358,JAN VERLEIJ,11945,26354,Testament,1761-06-24,nederlands,None,"[{'voornaam': 'Magteld', 'tussenvoegsel': None...","['KLAB06150000131.JPG', 'KLAB06150000132.JPG',...",160 Mgendes Dit Testament van Man No: 7160 en ...,Harmijntje ten Bosch,0,0,0
5003,7cbe357d-8773-4529-dafd-8aced1e59ed8,358,JAN VERLEIJ,11945,27395,Machtiging,1761-05-20,nederlands,\nSchip de Jonge Catharina\n,"[{'voornaam': 'Sicke', 'tussenvoegsel': None, ...","['KLAB06150000050.JPG', 'KLAB06150000051.JPG',...",Aatijs N �134 Procuratie Gepasseert den 20 Mai...,Sicke Jentjes,0,0,0
5004,7cbe357d-8773-4529-dafd-8aced1e59ed8,358,JAN VERLEIJ,11945,27395,Machtiging,1761-05-20,nederlands,\nSchip de Jonge Catharina\n,"[{'voornaam': 'Sicke', 'tussenvoegsel': None, ...","['KLAB06150000050.JPG', 'KLAB06150000051.JPG',...",Aatijs N �134 Procuratie Gepasseert den 20 Mai...,Abraham Mijlius,"[{'index': 298118, 'name_original': 'Abraham M...","{'298118': {'ships': [], 'rank': [], 'location...",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5606,ec86eb9e-7c9b-a20e-0ba9-31dc4e2d05eb,358,JAN VERLEIJ,11953,33691,Kwitantie,1764-01-19,nederlands,"\nSchip De Bregitta Antonia, crediteuren van o...","[{'voornaam': 'Andries', 'tussenvoegsel': None...","['KLAB06159000055.JPG', 'KLAB06159000056.JPG',...",No: 17 Quitantie gepasseert den 19e: Januarij ...,Maurits Kier,0,0,0
5607,ec86eb9e-7c9b-a20e-0ba9-31dc4e2d05eb,358,JAN VERLEIJ,11953,33691,Kwitantie,1764-01-19,nederlands,"\nSchip De Bregitta Antonia, crediteuren van o...","[{'voornaam': 'Andries', 'tussenvoegsel': None...","['KLAB06159000055.JPG', 'KLAB06159000056.JPG',...",No: 17 Quitantie gepasseert den 19e: Januarij ...,Jacob Fredrik Stijnman,"[{'index': 261563, 'name_original': 'Jacob Fre...","{'261563': {'ships': [], 'rank': [], 'location...",0
5608,ec86eb9e-7c9b-a20e-0ba9-31dc4e2d05eb,358,JAN VERLEIJ,11953,33691,Kwitantie,1764-01-19,nederlands,"\nSchip De Bregitta Antonia, crediteuren van o...","[{'voornaam': 'Andries', 'tussenvoegsel': None...","['KLAB06159000055.JPG', 'KLAB06159000056.JPG',...",No: 17 Quitantie gepasseert den 19e: Januarij ...,Hendrik Kannegieter Jansz,0,0,0
5609,ec86eb9e-7c9b-a20e-0ba9-31dc4e2d05eb,358,JAN VERLEIJ,11953,33691,Kwitantie,1764-01-19,nederlands,"\nSchip De Bregitta Antonia, crediteuren van o...","[{'voornaam': 'Andries', 'tussenvoegsel': None...","['KLAB06159000055.JPG', 'KLAB06159000056.JPG',...",No: 17 Quitantie gepasseert den 19e: Januarij ...,Racheltje Teunis,0,0,0


#### Correcties:
Chiel:
- Christiaan Andriesz 1416
- Jan de Bruijn 1463
- Abraham van de Heuvel (1823 super interessant twijfel geval)
- Pieter van Essen (1825 super interessant twijfel geval)
- Hans Pietersz 3792

Patrick:
- Jan Smit 4194 (zelde als de interessante gevallen van Chiel)
- Hendrik van Alen 593 (zelfde schip maar naam is net even anders)

In [44]:
subset1_pat.at[593, 'vocop_match'] = 0
subset1_chiel.at[1416,'vocop_match'] = 0
subset1_chiel.at[1463,'vocop_match'] = 0
subset2_chiel.at[3729, 'vocop_match'] = 0

In [45]:
final_pat = subset1_pat
final_pat = final_pat.append(subset2_pat)
final_pat = final_pat.append(subset3_pat)

final_chiel = subset1_chiel
final_chiel = final_chiel.append(subset2_chiel)
final_chiel = final_chiel.append(subset3_chiel)

final_barry = subset1_bar
final_barry = final_barry.append(subset2_bar)

final_df = final_pat.append(final_chiel).append(subset1_thom).append(final_barry)

In [46]:
final_df[final_df.vocop_match != 0]

,uuid,rubriek,notaris,inventarisNr,akteNr,akteType,datering,taal,beschrijving,namen,urls,text,name,data_matches,data_entry,vocop_match
372,2dd7b308-e622-01b5-22ae-abdeba00fed2,358,JAN VERLEIJ,11889,11294,Machtiging,1744-07-02,nederlands,\nschip Nieuland; schip Zaamslag; schip Waterv...,"[{'voornaam': 'Jurriaan', 'tussenvoegsel': Non...","['KLAB05459000335.JPG', 'KLAB05459000336.JPG',...",No: 259 Grseter gepasseert den 2e: Julij 1744J...,Christiaan Morgenfeld,"[{'index': 44997, 'name_original': 'Christiaan...","{'44997': {'ships': [], 'rank': [], 'location'...","[Christiaan Morgenfeld, 298101]"
375,2dd7b308-e622-01b5-22ae-abdeba00fed2,358,JAN VERLEIJ,11889,11294,Machtiging,1744-07-02,nederlands,\nschip Nieuland; schip Zaamslag; schip Waterv...,"[{'voornaam': 'Jurriaan', 'tussenvoegsel': Non...","['KLAB05459000335.JPG', 'KLAB05459000336.JPG',...",No: 259 Grseter gepasseert den 2e: Julij 1744J...,Nanning Broeder,"[{'index': 68473, 'name_original': 'Nanning Br...","{'68473': {'ships': ['Zaamslag', 'ZAAMSLAG'], ...","[Nanning Broeder, 68473]"
376,2dd7b308-e622-01b5-22ae-abdeba00fed2,358,JAN VERLEIJ,11889,11294,Machtiging,1744-07-02,nederlands,\nschip Nieuland; schip Zaamslag; schip Waterv...,"[{'voornaam': 'Jurriaan', 'tussenvoegsel': Non...","['KLAB05459000335.JPG', 'KLAB05459000336.JPG',...",No: 259 Grseter gepasseert den 2e: Julij 1744J...,Jan Fredrik Stephaan,"[{'index': 256327, 'name_original': 'Jan Fredr...","{'256327': {'ships': [], 'rank': [], 'location...","[Jan Fredrik Stephaen, 289548]"
509,ee21c8bc-bfea-25ee-c6c1-ddf74644012b,358,JAN VERLEIJ,11890,12427,Obligatie,1744-09-12,nederlands,\nschip Diemen\n,"[{'voornaam': 'Pieter', 'tussenvoegsel': None,...","['KLAB05439000262.JPG', 'KLAB05439000263.JPG',...",Ragd werlke Getijnke Gepasseert Den 12 feptemb...,Pieter Klijn,"[{'index': 2148, 'name_original': 'Pieter Clin...","{'2148': {'ships': [], 'rank': [], 'location':...","[Pieter Klijn, 284049]"
563,35b32066-d1ee-0b29-2180-38584da36f12,358,JAN VERLEIJ,11891,11348,Obligatie,1744-10-29,nederlands,"\nschip d' Anna, VOC, 300 gulden\n","[{'voornaam': 'Nomme', 'tussenvoegsel': None, ...","['KLAB05426000116.JPG', 'KLAB05426000117.JPG',...",No: 636 Puijete Guijfiet Den 29 Eebber 1744Nte...,Cornelis Christiaansz,"[{'index': 13740, 'name_original': 'Cornelis C...","{'13740': {'ships': [], 'rank': [], 'location'...","[Cornelis Christiaansz, 329326]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54466,da6f9044-1588-3bf2-1e2f-72b7b374e3db,358,JAN VERLEIJ,11905,18207,Machtiging,1748-09-19,nederlands,"\nschip 't Huys te Spijk, VOC kamer Zeeland; s...","[{'voornaam': 'Cornelis', 'tussenvoegsel': Non...","['KLAB05559000102.JPG', 'KLAB05559000103.JPG']",No: 259 Procuratie gepoassseert den 19e: Septe...,Jan Govertsz,"[{'index': 21034, 'name_original': 'Jan Govers...","{'21034': {'ships': [], 'rank': [], 'location'...","[Jan Govertsz., 66979]"
54595,e38f4bb1-65cb-0a19-4bfc-7da44f89b3d8,358,JAN VERLEIJ,11906,20281,Machtiging,1749-02-11,nederlands,\nschip Diemen; schip d'Anna (kamer Zeeland)\n,"[{'voornaam': 'Johan', 'tussenvoegsel': None, ...","['KLAB05516000182.JPG', 'KLAB05516000183.JPG',...",�� 25:8:No: 56 Grvretie 5 en de Je: HrijverJra...,Hendrik Burger,"[{'index': 3138, 'name_original': 'Hendrick Bu...","{'3138': {'ships': [], 'rank': [], 'location':...","[Hendrik Burgers, 297772]"
54707,3200f9a9-3a8f-c498-e9fd-a5d96430fb72,358,JAN VERLEIJ,11908,12939,Machtiging,1749-08-28,nederlands,\nzaakwaarneming afwezigheid; schip Sloterdijk\n,"[{'voornaam': 'Cornelis', 'tussenvoegsel': Non...","['KLAB05596000339.JPG', 'KLAB05596000340.JPG',...",No: 411 Procuratie gecasseerd den 28e: ang: 17...,Cornelis Oberg,"[{'index': 114308, 'name_original': 'Cornelis ...","{'114308': {'ships': [], 'rank': [], 'location...","[Cornelis Oberg, 367003]"
54741,7dff1bd9-39c9-c90f-d687-ae77a702b3f1,358,JAN VERLEIJ,11908,17597,Testament,1749-08-19,nederlands,\nschip 't Kasteel van Tilburg\n,"[{'voornaam': 'Jacobus', 'tussenvoegsel': None...","['KLA